In [4]:
import pandas as pd
import time
import numpy as np
import re
import pickle
import string, unicodedata
from unidecode import unidecode

In [5]:
a = time.time()

## Load Data

In [6]:
#data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'Sheet1') # data awl
data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'imbang') #imbang
#data = pd.read_excel('data/data_komentar_clean_1.xlsx', sheet_name = 'imb_7525')
komentar = data['komentar'].tolist()
label = data['label'].tolist()
kode = data['kode'].tolist()

In [3]:
len(komentar)

3400

## Proses TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
      
tfidf = TfidfVectorizer() #membuat model tf-idf
##Membuat Matrix TF-IDF
tfidf_matrix = tfidf.fit_transform(komentar) #melakukan fit data -> seluruh data komentar
print(tfidf_matrix.shape)
X = tfidf_matrix.toarray() #merubah tipe data ke array
#X= tfidf_matrix
Y = np.array(kode) #labet tiap (komentat [spam dan bukan spam])/ 1:0
y = Y

(3400, 12215)


## Proses Pemodelan

In [8]:
#Import Library
import numpy as np
from sklearn import svm
from sklearn.naive_bayes import ComplementNB

import pylab as pl
from sklearn.svm import SVC

# Mencari parameter terbaik

## Proposes percobaan untuk masing masing parameter SVM

In [ ]:
K = 10 #pembagian data uji dan data latih
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=K) 
C_range = np.arange(1.0,2.6,0.1)
gamma_range = np.arange(0.1,1.1, 0.1)

C_range_ = []
gamma_range_ = []
rerata_akurasi_svm = []
rerata_waktu_eksekusi = []
rerata_waktu_prediksi = []
count=0
for i in range(len(C_range)):
    for j in range(len(gamma_range)):
        C = C_range[i]
        gamma = gamma_range[j]
        akurasi_svm = []
        waktu_eksekusi = []
        waktu_prediksi = []
        cSVM = svm.SVC(kernel='rbf', gamma=gamma, C=C) #model
        for train_index, test_index in kf.split(X):
            #membagi data latih dan data uji
            X_train, X_test = X[train_index], X[test_index] #pembagian data latih
            y_train, y_test = y[train_index], y[test_index]
            
            start = time.time()
            #Melatih Model
            cSVM = cSVM.fit(X_train, y_train) #SVM
            end = time.time()
            waktu_eksekusi.append(end-start)
            
            #proses prediksi
            start = time.time()
            P_SVM = cSVM.predict(X_test)
            end = time.time()
            waktu_prediksi.append(end-start)
            
            akurasi_svm.append(accuracy_score(y_test, P_SVM))
        print("iterasi ke-", count)
        C_range_.append(C)
        gamma_range_.append(gamma)
        
        rr = sum(akurasi_svm)/len(akurasi_svm)
        rerata_akurasi_svm.append(rr)
        count+=1
        
        rerata_waktu_eksekusi.append(sum(waktu_eksekusi)/len(waktu_eksekusi))
        rerata_waktu_prediksi.append(sum(waktu_prediksi)/len(waktu_prediksi))
        
        #Menampilkan hasil 
        print("-"*30)
        print("C     : ", C)
        print("gamma : ", gamma)
        print("-"*20)
        print("Rata-rata Akurasi SVM : %0.2f" % (sum(akurasi_svm)/len(akurasi_svm)))
        print("Rata-rata Waktu eksekusi SVM: %0.2f" % (sum(waktu_eksekusi)/len(waktu_eksekusi)))
        print("Rata-rata Waktu prediksi  SVM: %0.2f" % (sum(waktu_prediksi)/len(waktu_prediksi)))
        print("="*30)
    
#proses mencari parameter dan akurasi terbaik
x = max(rerata_akurasi_svm)
#al = list(alpha_range)
index = rerata_akurasi_svm.index(x)
round(gamma_range_[index],2)
print("---------------------------")
print("gamma terbaik adalah :", round(gamma_range_[index],2))
print("    C terbaik adalah :", round(C_range_[index],2))
print("             Akurasi :", round(x, 2))

#MENYIMPAN HASIL
dict_komentar  = {"gamma":gamma_range_,"c":C_range_, "akurasi":rerata_akurasi_svm, "waktu eksekusi":rerata_waktu_eksekusi, "waktu prediksi":rerata_waktu_prediksi}
df = pd.DataFrame(dict_komentar,columns = ["gamma", 'c', "akurasi", "waktu eksekusi", "waktu prediksi"])
writer = pd.ExcelWriter('data/parameter-akurasi-svm_7525_Time_balance.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='svm-parameter-svm')
#Close the Pandas Excel writer and output the Excel file.
writer.save()

iterasi ke- 0
------------------------------
C     :  1.0
gamma :  0.1
--------------------
Rata-rata Akurasi SVM : 0.88
Rata-rata Waktu eksekusi SVM: 157.48
Rata-rata Waktu prediksi  SVM: 17.27
iterasi ke- 1
------------------------------
C     :  1.0
gamma :  0.2
--------------------
Rata-rata Akurasi SVM : 0.90
Rata-rata Waktu eksekusi SVM: 142.84
Rata-rata Waktu prediksi  SVM: 16.05
iterasi ke- 2
------------------------------
C     :  1.0
gamma :  0.30000000000000004
--------------------
Rata-rata Akurasi SVM : 0.91
Rata-rata Waktu eksekusi SVM: 138.82
Rata-rata Waktu prediksi  SVM: 15.50
iterasi ke- 3
------------------------------
C     :  1.0
gamma :  0.4
--------------------
Rata-rata Akurasi SVM : 0.91
Rata-rata Waktu eksekusi SVM: 136.25
Rata-rata Waktu prediksi  SVM: 15.44


# Mencari Parameter Alpha

In [9]:
K = 10 #pembagian data uji dan data latih
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score#, precision_score

kf = KFold(n_splits=K)

alpha_range = np.arange(0.1,2.01,0.01)

rerata_akurasi_cnb = []
rerata_waktu_eksekusi = []
rerata_waktu_prediksi = []

count=0
for i in range(len(alpha_range)): #melakukan interasi sebanyak data parameter C
    alpha = alpha_range[i]
    akurasi_cnb = []
    comNB = ComplementNB(alpha = alpha) #model
    waktu_eksekusi = []
    waktu_prediksi = []
    tic = time.perf_counter()
    for train_index, test_index in kf.split(X):
            #membagi data latih dan data uji
        X_train, X_test = X[train_index], X[test_index] #pembagian data latih
        y_train, y_test = y[train_index], y[test_index]
    
        #Melatih Model 
        mCNB = comNB.fit(X_train, y_train) 
            
        #proses prediksi
        P_CNB = mCNB.predict(X_test)
        
        
        start = time.time()
        #Melatih Model
        mCNB = comNB.fit(X_train, y_train)  #SVM
        end = time.time()
        waktu_eksekusi.append(end-start)
            
        #proses prediksi
        start = time.time()
        P_CNB = mCNB.predict(X_test)
        end = time.time()
        waktu_prediksi.append(end-start)
        
        
        akurasi_cnb.append(accuracy_score(y_test, P_CNB))
    toc = time.perf_counter()
    print("iterasi ke-", count)    
    rr = sum(akurasi_cnb)/len(akurasi_cnb)
    rerata_akurasi_cnb.append(rr)
    
    rerata_waktu_eksekusi.append(sum(waktu_eksekusi)/len(waktu_eksekusi))
    rerata_waktu_prediksi.append(sum(waktu_prediksi)/len(waktu_prediksi))
    count+=1
    #Menampilkan hasil akhir
    print("---------------------------------")
    print("alpha : ", alpha)
    print("----------------------------")
    print("Rata-rata Akurasi cnb: %0.2f" % (sum(akurasi_cnb)/len(akurasi_cnb)))
    print("---------------------------------", toc-tic, "second")
    
#proses mencari parameter dan akurasi terbaik
x = max(rerata_akurasi_cnb)
index = rerata_akurasi_cnb.index(x)
print("---------------------------")
print("alpha terbaik adalah :", round(alpha_range[index],2))
print("             Akurasi :", round(x, 2))

round_akurasi_cnb = []
for i in rerata_akurasi_cnb:
    round_akurasi_cnb.append(round(i,2))
dict_komentar  = {"alpha":alpha_range,"akurasi":rerata_akurasi_cnb, "Akurasi round":round_akurasi_cnb, "waktu eksekusi":rerata_waktu_eksekusi, "waktu prediksi":rerata_waktu_prediksi}
df = pd.DataFrame(dict_komentar,columns = ["alpha","akurasi","Akurasi round", "waktu eksekusi", "waktu prediksi"])
writer = pd.ExcelWriter('data/parameter-akurasi-cnb_7525_time.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='cnb-parameter-cnb')

#menyimpan file
writer.save()

iterasi ke- 0
---------------------------------
alpha :  0.1
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.4333659 second
iterasi ke- 1
---------------------------------
alpha :  0.11
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.050822899999998 second
iterasi ke- 2
---------------------------------
alpha :  0.12
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.071719499999997 second
iterasi ke- 3
---------------------------------
alpha :  0.13
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.029397200000002 second
iterasi ke- 4
---------------------------------
alpha :  0.13999999999999999
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.429783100000002 second
iterasi ke- 5
---------------------------------
alpha :  0.14999999999999997
---------------------------

iterasi ke- 43
---------------------------------
alpha :  0.5299999999999998
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.100771399999985 second
iterasi ke- 44
---------------------------------
alpha :  0.5399999999999998
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.135681199999965 second
iterasi ke- 45
---------------------------------
alpha :  0.5499999999999998
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.153075400000034 second
iterasi ke- 46
---------------------------------
alpha :  0.5599999999999997
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.181441900000038 second
iterasi ke- 47
---------------------------------
alpha :  0.5699999999999997
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 9.356762099999969 second
iterasi ke- 48
--------------------

iterasi ke- 86
---------------------------------
alpha :  0.9599999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.11155610000003 second
iterasi ke- 87
---------------------------------
alpha :  0.9699999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.444974600000023 second
iterasi ke- 88
---------------------------------
alpha :  0.9799999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.084953499999983 second
iterasi ke- 89
---------------------------------
alpha :  0.9899999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.069482300000004 second
iterasi ke- 90
---------------------------------
alpha :  0.9999999999999996
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.046301200000016 second
iterasi ke- 91
---------------------

iterasi ke- 129
---------------------------------
alpha :  1.3899999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.428998400000069 second
iterasi ke- 130
---------------------------------
alpha :  1.3999999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.319123499999932 second
iterasi ke- 131
---------------------------------
alpha :  1.4099999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 6.8743445000000065 second
iterasi ke- 132
---------------------------------
alpha :  1.4199999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 6.793433299999833 second
iterasi ke- 133
---------------------------------
alpha :  1.4299999999999995
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 6.8417475000001104 second
iterasi ke- 134
------------

iterasi ke- 172
---------------------------------
alpha :  1.8199999999999992
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.726106200000004 second
iterasi ke- 173
---------------------------------
alpha :  1.8299999999999992
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.127719900000102 second
iterasi ke- 174
---------------------------------
alpha :  1.8399999999999992
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.200412200000073 second
iterasi ke- 175
---------------------------------
alpha :  1.8499999999999992
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.0950390999998945 second
iterasi ke- 176
---------------------------------
alpha :  1.8599999999999992
----------------------------
Rata-rata Akurasi cnb: 0.92
--------------------------------- 7.096752800000104 second
iterasi ke- 177
-------------